In [1]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import re

In [2]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

In [3]:
#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
certificate = []
time=[]
genre = []
rating=[]
metascore =[]
votes = []
gross = []
description = []

In [4]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [5]:
#the whole core of the script
for page in pages:
    
    if(len(movie_name) >= 10):
        break
    
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt", headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        
        if(len(movie_name) >= 10):
            break
        
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(re.findall("\d+", year_of_release)[0])
        
        certificate_rating = store.p.find("span", class_ = 'certificate').text if store.p.find("span", class_ = 'certificate') else np.nan
        certificate.append(certificate_rating)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.p.find("span", class_ = 'runtime') else np.nan
        runtime = np.nan if runtime is np.nan else runtime[:len(runtime) - 4]
        time.append(runtime)
        
        movie_genre = store.p.find("span", class_ = 'genre').text.replace('\n', '') if store.p.find("span", class_ = 'genre') else np.nan
        genre.append(movie_genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else np.nan
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text.replace(',', '')
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 and re.match("\$(\d+?.?\d+?)M", value[1].text) else np.nan
        grosses = np.nan if grosses is np.nan else grosses[1:len(grosses) - 1]
        gross.append(grosses)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else np.nan
        description.append(description_)

In [6]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie_Name": movie_name, "Year_of_Release" : year, "C_Rating": certificate,  "Watch_Time_(min)": time, "Genre": genre , "Movie_Rating": rating, "Metascore": metascore, "Votes" : votes, "Gross_($M)": gross, "Description": description  })

In [7]:
movie_list.head(10)

,Movie_Name,Year_of_Release,C_Rating,Watch_Time_(min),Genre,Movie_Rating,Metascore,Votes,Gross_($M),Description
0,The Shawshank Redemption,1994,R,142,Drama,9.3,82,2740129,28.34,"Over the course of several years, two convicts..."
1,The Godfather,1972,R,175,"Crime, Drama",9.2,100,1905483,134.97,"Don Vito Corleone, head of a mafia family, dec..."
2,The Dark Knight,2008,PG-13,152,"Action, Crime, Drama",9.0,84,2713094,534.86,When the menace known as the Joker wreaks havo...
3,Schindler's List,1993,R,195,"Biography, Drama, History",9.0,95,1382866,96.90,"In German-occupied Poland during World War II,..."
4,The Godfather Part II,1974,R,202,"Crime, Drama",9.0,90,1298675,57.30,The early life and career of Vito Corleone in ...
5,12 Angry Men,1957,Approved,96,"Crime, Drama",9.0,97,811070,4.36,The jury in a New York City murder trial is fr...
6,The Lord of the Rings: The Return of the King,2003,PG-13,201,"Action, Adventure, Drama",9.0,94,1884015,377.85,Gandalf and Aragorn lead the World of Men agai...
7,Pulp Fiction,1994,R,154,"Crime, Drama",8.9,95,2105619,107.93,"The lives of two mob hitmen, a boxer, a gangst..."
8,Inception,2010,PG-13,148,"Action, Adventure, Sci-Fi",8.8,74,2408321,292.58,A thief who steals corporate secrets through t...
9,The Lord of the Rings: The Fellowship of the Ring,2001,PG-13,178,"Action, Adventure, Drama",8.8,92,1912764,315.54,A meek Hobbit from the Shire and eight compani...
